# API development for download of Sentinel-2 and Landsat-8 data
### User defined mosaicing of harmonised products
_Robin Kohrs & Konstantin Schellenberg, February 2020, GEO 419_  
_Supervisor: John Truckenbrodt¹, Martin Habermeyer²_ <br>
<sub><sub>
¹ Friedrich-Schiller-University Jena, chair of remote sensing <br>
² Deutsches Luft- und Raumfahrtszentrum, earth observation center
</sub></sub>

## Tutorial
This is the extention for nasa_hls. We developed following new functions:

In order to make the program be more flexible, the goal of this extension is to
1. Download tiles by setting spatial (user defined verctor geometry) and temporal inqueries to the HLS server,
2. Mosaic the HDF4 formatted product per band and acquisition date.

As part of the module report will want to add the following post procession tasks: 
3. Calculate spectral indexes from the downloaded products.
4. Plot the results inline with `ipyleafet` and `folium` on a Open Street Map WMS service.

Tasks 1 and 2 are meant to work in accordance with the `nasa_hls` package and aims to lead to a pull request to the repository; 3 and 4 shall give an overview of to use the new utilities and the HLS product.

User guide to the HLS product:
https://hls.gsfc.nasa.gov/wp-content/uploads/2019/01/HLS.v1.4.UserGuide_draft_ver3.1.pdf

## <font color = "red"> Download HSL files with user input</font>

In [1]:
# change working directory 
import os
os.chdir("/home/aleko-kon/projects/nasa_hls")

In [2]:
import warnings
warnings.simplefilter("ignore")

In [3]:
%matplotlib inline

import nasa_hls
import sys
import pandas as pd
import os
from osgeo import gdal
import geopandas as gpd
from fiona.crs import from_epsg
gdal.UseExceptions()

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# and for later processing in the notebook

- For testing purpose, try downloading the kml file:

In [1]:
nasa_hls.download_kml()

NameError: name 'nasa_hls' is not defined

## <font color = "red">Make list with GeoDataFrames</font>

- This will serve as a conformation for the user
- It's a list of lists. Each inner list beeing the GeoDataFrames for the specified dates

In [5]:
# define user shape
shape_path="/home/aleko-kon/Dokumente/nasa_hls/data/dummy_region.shp"

In [1]:
ds = nasa_hls.make_tiles_dataset(shape=shape_path,
                                products=["L30"],
                                start_date="2018-12-25")


NameError: name 'nasa_hls' is not defined

In [7]:
# print the list of lists
print(ds, "\n\n")
print("The query starting at date '2018-12-25', resulted in {len} hls-prodcuts".format(len = len(ds)))


[    product   tile       date                                                url
44      L30  34JDN 2018-12-25  https://hls.gsfc.nasa.gov/data/v1.4/L30/2018/3...
71      L30  34JEN 2018-12-25  https://hls.gsfc.nasa.gov/data/v1.4/L30/2018/3...
116     L30  34JDP 2018-12-25  https://hls.gsfc.nasa.gov/data/v1.4/L30/2018/3...
143     L30  34JEP 2018-12-25  https://hls.gsfc.nasa.gov/data/v1.4/L30/2018/3...] 


The query starting at date '2018-12-25', resulted in 1 hls-prodcuts


- As one can see in the list, there are 4 scenes for the 27th of december and two products for the 29th.

***
## <font color = "red">Download Tiles</font>

- Taking the list from above, the data sources can be downloaded via the function `download_tiles`. 

- This function calls `download_batch` and other methods internally in order to parse the right URLs for download.

- The path for the parameter `dir` needs to be adapted, as this is user-specific.

In [8]:
# where the user wants the ".hdf"-files
hdf_path = "/home/aleko-kon/Dokumente/nasa_hls/data/hdf2"

In [9]:
nasa_hls.download_tiles(dstdir=hdf_path, dataframes=ds)

100%|██████████| 4/4 [04:36<00:00, 69.11s/it]


In [10]:
# the size of one ".hdf"-file
s = os.path.getsize(os.path.join(hdf_path , "HLS.S30.T34JDN.2018361.v1.4.hdf"))
print("The first downloaded 'hdf'-file is {0:.2f} MB in size on the disk".format(s/1e6))

The first downloaded 'hdf'-file is 244.24 MB in size on the disk


***
## <font color = "red">Mosaicing tiles</font>
- Now we'll create a mosaic for each day. Internally there are two steps. First creating a mosaic for each band for each day. And afterwards creating a mosaic for each day of all the single bands.
- This is done by the function `make_mosaic`.

In [1]:
# for simplicity make input and output file directories the same
# tiffs will end up in the same directory
# note the "/" at the end

%matplotlib inline

import nasa_hls
import sys
import pandas as pd
import os
from osgeo import gdal
import geopandas as gpd
from fiona.crs import from_epsg
gdal.UseExceptions()
from nasa_hls.download_tiles import path_auxil
import shutil

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


final_dir = "/home/aleko-kon/Dokumente/nasa_hls/data/out/"
shape_path="/home/aleko-kon/Dokumente/nasa_hls/data/dummy_region.shp"
hdf_path = "/home/aleko-kon/Dokumente/nasa_hls/data/mixed/"

print(final_dir)
print(hdf_path)
print(path_auxil)


/home/aleko-kon/Dokumente/nasa_hls/data/out/
/home/aleko-kon/Dokumente/nasa_hls/data/mixed/
/home/aleko-kon/.nasa_hls/.auxdata/


In [3]:
nasa_hls.make_mosaic(srcdir=hdf_path, 
                     dstdir=final_dir, 
                     product="L30")

Landsat
Final VRT: 
 /home/aleko-kon/.nasa_hls/.auxdata/mosaic/days/359final.vrt
Outfile: 
 /home/aleko-kon/Dokumente/nasa_hls/data/out/359.tif 



***
## <font color = "red">Print Results</font>

In [10]:
import rasterio
import matplotlib.pyplot as plt
import matplotlib as mpl
import fiona
import numpy as np
import rasterio as rio
from rasterio.plot import show
from rasterio.plot import show_hist
from descartes import PolygonPatch

In [ ]:
#filepath of one of the rasters (hdf-scenes)
fp = "/home/robin/python_projects/data/nasa_hls/hdf_tiles/361.tiff"

In [ ]:
rio_raster = rio.open(fp) #produces rasterio.DatasetReader
print(type(rio_raster) )
print(rio_raster.bounds)

In [ ]:
with rio.open(fp) as src:
    data = src.read() # produces numpy nd array
type(data)

In [ ]:
# Confirm that there are 12 bands for Sentinel Scenes
data.shape

In [ ]:
# show first band cropped to our shapefile
show((data[1,:,:]))

In [ ]:
# get the entire (uncropped) raster 
fp_vrt = "/home/robin/.nasa_hls/.auxdata/mosaic/days/361final.vrt"
raster_vrt = rio.open(fp_vrt)

#and plot both together
fig, ax = plt.subplots()
show(raster_vrt, ax = ax)
show(rio_raster, cmap = "pink", ax = ax)


In [ ]:
# now add the shape
shape = gpd.read_file(shape_path)
print(shape.crs)
shape = shape.to_crs("EPSG:32634")
print(shape.crs)

In [ ]:
#and plot both together
fig, ax = plt.subplots()
shape.plot(ax = ax, alpha=.5)
show(raster_vrt, ax = ax)
show(rio_raster, cmap = "pink", ax = ax)
ax.set_title("User-shape, full hls-scene and cropped scene")


### get some metadata

In [ ]:
# get the dimension of the cropped raster by reading the first band
first_band_array = rio_raster.read(1)
first_band_array_uncropped = raster_vrt.read(1)
print("the cropped rasters dimension is \n", first_band_array.shape)
print("")
print("the uncropped rasters dimension is \n", first_band_array_uncropped.shape)

In [ ]:
show_hist((rio_raster, 1), lw = 0.0)

In [ ]:
# when plotted as numpy array_first_band you can see the nodata-values of -1000
plt.hist(first_band_array, bins = 10)
plt.show()

In [ ]:
#how many values with -1000
sum_neg = (first_band_array == -1000).sum()
sum_neg 

In [ ]:
# what are  datatype and the nodata-values in the final tiff
for i, dtype, nodataval in zip(rio_raster.indexes, rio_raster.dtypes, rio_raster.nodatavals):
    print(i, dtype, nodataval)

In [ ]:
# what are  datatype and the nodata-values in the vrt, that makes the final tiff above?
for i, dtype, nodataval in zip(raster_vrt.indexes, raster_vrt.dtypes, raster_vrt.nodatavals):
    print(i, dtype, nodataval)

- As one can see there is one band that is just of datatype `uint8` instad of being `int16`. This is the QA-Band with a nodata-value of 255


In [ ]:
fig, (axr, axg, axb) = plt.subplots(1,3, figsize=(21,7))
show((rio_raster,4), ax = axr, cmap = "Reds", title = "red channel")
show((rio_raster,3), ax = axg, cmap = "Greens", title = "green channel")
show((rio_raster,2), ax = axb, cmap = "Blues", title = "blue channel" )


In [ ]:
red = rio_raster.read(4)
green = rio_raster.read(3)
blue = rio_raster.read(2)

In [ ]:
# convert -1000 to NAN
first_band_array = first_band_array.astype("float")
first_band_array[first_band_array == -1000] = np.NAN

In [ ]:
def normalize(array):
    """Normalizes numpy arrays into scale 0.0 - 1.0"""
    array_min, array_max = array.min(), array.max()
    return ((array - array_min)/(array_max - array_min))

redn = normalize(red)
greenn = normalize(green)
bluen = normalize(blue)


print("Normalized bands")
print(redn.min(), '-', redn.max(), 'mean:', redn.mean())
print(greenn.min(), '-', greenn.max(), 'mean:', greenn.mean())
print(bluen.min(), '-', bluen.max(), 'mean:', bluen.mean())


In [ ]:
# stack bands together
rgb = np.dstack((redn, greenn, bluen))
plt.imshow(rgb)

In [ ]:
import folium
from folium import plugins


In [ ]:
from rasterio.warp import calculate_default_transform, reproject, Resampling

In [ ]:
# Create variables for destination coordinate system and the name of the projected raster
dst_crs = 'EPSG:4326' 
out_path = os.path.join("/home/robin/python_projects/data/nasa_hls/hdf_tiles/361reproj.tiff")

In [ ]:
with rio.open(fp) as src:
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds)
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })
    
    with rio.open(out_path, 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
            source=rio.band(src, i),
            destination=rio.band(dst, i),
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=transform,
            dst_crs=dst_crs,
            resampling=Resampling.nearest)
            

In [ ]:
# Use rasterio to import the reprojected data as img
with rio.open(out_path) as src:
    boundary = src.bounds
    img = src.read()
    nodata = src.nodata

In [ ]:
boundary


In [ ]:
map_= folium.Map(location=[20, -29],
                  tiles='Stamen Terrain', zoom_start = 8)

In [ ]:
img = folium.raster_layers.ImageOverlay(
        name='Mercator projection SW',
        image=img[0],
        bounds=[[19.96, -29.91], [22.13, -28.02]],
        opacity=0.6,
        interactive=True,
        zindex=1,
    )

In [ ]:
img.add_to(map_)

In [ ]:
map_

## Calculate Indexes